In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import requests
from bs4 import BeautifulSoup
import time

In [55]:
data = pd.read_csv('houses_current.csv')
data.head()

,houseTypes,area,num_floors,bedrooms,address,price,url
0,Bán Nhà riêng,130.0,3.0,7.0,"Quận 12, TP Hồ Chí Minh",9.200,https://batdongsan.vn/ban-biet-thu-san-vuon-qu...
1,Bán Nhà riêng,65.0,NaN,2.0,"Ngũ Hành Sơn, Đà Nẵng",2.900,https://batdongsan.vn/29-ty-dien-tich-6580m2-n...
2,Bán Nhà riêng,90.0,5.0,5.0,"Hai Bà Trưng, Hà Nội",14.000,https://batdongsan.vn/ban-nha-kim-nguu-phan-lo...
3,Bán Nhà riêng,50.0,5.0,5.0,"Quận 3, TP Hồ Chí Minh",0.001,https://batdongsan.vn/quan-3-le-van-sy-5-tang-...
4,Bán Nhà riêng,30.0,1.0,2.0,"Gò Vấp, TP Hồ Chí Minh",3.700,https://batdongsan.vn/nha-moi-full-noi-that-hx...


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9674 entries, 0 to 9673
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   houseTypes  9674 non-null   object 
 1   area        9497 non-null   float64
 2   num_floors  7706 non-null   float64
 3   bedrooms    7577 non-null   float64
 4   address     9674 non-null   object 
 5   price       9548 non-null   float64
 6   url         9674 non-null   object 
dtypes: float64(4), object(3)
memory usage: 529.2+ KB


REMOVE DUPLICATES

In [56]:
print('Sum of duplicates: ', data.duplicated(subset=['houseTypes', 'area', 'num_floors', 'bedrooms','address','price']).sum())
data = data.drop_duplicates(subset=['houseTypes', 'area', 'num_floors', 'bedrooms','address','price']).copy()
print('After removing duplicates: ', data.duplicated().sum())
print('Length of data then: ', len(data))

Sum of duplicates:  1066
After removing duplicates:  0
Length of data then:  8608


CONVERT OBJECT TO FLOAT64 AND STRING

In [57]:
data = data.astype({'houseTypes':'string','address':'string','url':'string'})

In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8608 entries, 0 to 9673
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   houseTypes  8608 non-null   string 
 1   area        8447 non-null   float64
 2   num_floors  6798 non-null   float64
 3   bedrooms    6698 non-null   float64
 4   address     8608 non-null   string 
 5   price       8492 non-null   float64
 6   url         8608 non-null   string 
dtypes: float64(4), string(3)
memory usage: 538.0 KB


In [59]:
for col in data.columns:
    print('Ratio of missing data of ' + str(col) +': ' + str(round(data[col].isna().sum()*100/len(data[col]),3)) + '%')

Ratio of missing data of houseTypes: 0.0%
Ratio of missing data of area: 1.87%
Ratio of missing data of num_floors: 21.027%
Ratio of missing data of bedrooms: 22.189%
Ratio of missing data of address: 0.0%
Ratio of missing data of price: 1.348%
Ratio of missing data of url: 0.0%


In [60]:
data[['district','city']] = data['address'].apply(lambda x: pd.Series(str(x).split(', ')))

In [61]:
data = data.drop(['address'], axis=True)

In [62]:
data.reset_index(inplace=True, drop=True)

In [63]:
data = data[['houseTypes','area','num_floors','bedrooms','district','city','price','url']]

In [64]:
data.head()

,houseTypes,area,num_floors,bedrooms,district,city,price,url
0,Bán Nhà riêng,130.0,3.0,7.0,Quận 12,TP Hồ Chí Minh,9.200,https://batdongsan.vn/ban-biet-thu-san-vuon-qu...
1,Bán Nhà riêng,65.0,NaN,2.0,Ngũ Hành Sơn,Đà Nẵng,2.900,https://batdongsan.vn/29-ty-dien-tich-6580m2-n...
2,Bán Nhà riêng,90.0,5.0,5.0,Hai Bà Trưng,Hà Nội,14.000,https://batdongsan.vn/ban-nha-kim-nguu-phan-lo...
3,Bán Nhà riêng,50.0,5.0,5.0,Quận 3,TP Hồ Chí Minh,0.001,https://batdongsan.vn/quan-3-le-van-sy-5-tang-...
4,Bán Nhà riêng,30.0,1.0,2.0,Gò Vấp,TP Hồ Chí Minh,3.700,https://batdongsan.vn/nha-moi-full-noi-that-hx...


In [65]:
data.to_csv('houses.csv',index=False)
data.to_excel('houses.xlsx',index=False)

In [50]:
low_noise_data = data.loc[(data['price'] <= 0.5)]
hign_noise_data = data.loc[(data['price'] >= 500)]
null_price_data = data[data['price'].isna()]


In [66]:
low_noise_data = low_noise_data.copy()

In [68]:
new_low_price = []
for url in low_noise_data['url']:
    match = re.search(r'(\d+)-ty', url) or re.search(r'(\d+)ty-', url) or re.search(r'(\d+)-ti', url)
    #   or re.search(r'(\d+)-toi', url)
    if match:
        number = match.group(1)
        if int(number) <= 10:
            new_low_price.append(float(number))
        elif int(number) <= 100:
            new_low_price.append(float(number)/10)
        else:
            new_low_price.append(float(number)/100)
    else:
        match = re.search(r'(\d+)-trieu-', url) or re.search(r'(\d+)tr-', url) or re.search(r'(\d+)trieu-', url)
        if match:
            number = match.group(1)
            new_low_price.append(float(number)/1000)
        else:
            new_low_price.append(0)
  
# -2ty5-    -70trm2-    -2x-ty-  -4ty95-    -2ty2x-     -shr6ty120-  -360trphan             

In [ ]:
# def recollecting_price():
    

In [ ]:
len(new_low_price)
low_noise_data['new_price'] = new_low_price                                     
low_noise_data.to_excel('a.xlsx', index=False)  

In [91]:
new_price =[]
count_error = 0
for url in low_noise_data['url']:
    match = re.search(r'(\d+)-ty', url) 
    if match:
        number = match.group(1)
        if int(number) > 10 & int(number) < 100:
            new_price.append(float(number)/10)
        elif int(number) >= 100:
            new_price.append(float(number)/100)
        else:
            new_price.append(float(number))
    else:
        match =  re.search(r'(\d+)tr', url)
        if match:
            new_price.append(float(match.group(1))/1000)
        elif match == None:
            try:
                data = requests.get(url)
                soup = BeautifulSoup(data.text, 'lxml')
                if data:
                    if(soup.find('div',{'class':'project-global-object-block-003 information-custom'}) and soup.find('div',{'class':'content'})):
                        text_info=""
                        text_info = soup.find('div',{'class':'project-global-object-block-003 information-custom'}).text.strip() + soup.find('div',{'class':'content'}).text.strip().replace('\n','')
                        text_info = text_info.replace("\r"," ")
                        text_info = text_info.replace("\n"," ")
                        text_info = text_info.replace("\t"," ")
                        z = re.search(r"(\d+(?:\.\d+)?(?:,\d+)?) Tỷ", text_info, re.IGNORECASE)
                        if z:
                            new_price.append(float(z.group(1)))
            except:
                print("error")
                new_price.append(0.00001)
        else:
            new_price.append(0.00001)

len(low_noise_data)
#low_noise_data['new_price'] = new_price
new_price

error
error
error
error


[41.5,
 4.5,
 0.4,
 5.2,
 5.3,
 5.5,
 5.2,
 5.8,
 0.3,
 3.5,
 0.4,
 3.3,
 5.8,
 0.07,
 79.9,
 4.8,
 4.5,
 5.7,
 9.6,
 0.6,
 32.5,
 4.5,
 23.5,
 18.8,
 9.9,
 6.7,
 5.3,
 3.9,
 48.5,
 19.5,
 41.5,
 18.8,
 4.7,
 7.9,
 108.5,
 0.3,
 12.5,
 3.3,
 7.1,
 0.4,
 28.5,
 0.6,
 4.1,
 4.5,
 9.9,
 0.3,
 2.2,
 0.5,
 0.4,
 1.2,
 4.3,
 0.3,
 4.2,
 8.5,
 47.5,
 4.5,
 49.5,
 0.5,
 8.6,
 0.3,
 0.4,
 3.8,
 0.6,
 2.7,
 0.75,
 0.5,
 0.1,
 0.025,
 0.45,
 1e-05,
 0.035,
 0.088,
 0.075,
 0.5,
 0.33,
 0.4,
 5.0,
 0.47,
 10.0,
 229.0,
 0.04,
 0.5,
 6.7,
 49.5,
 0.5,
 8.7,
 0.95,
 1.6,
 0.48,
 36.8,
 4.0,
 2.0,
 0.45,
 0.5,
 387000.0,
 234000.0,
 2.15,
 0.4,
 0.43,
 0.3,
 0.36,
 3.3,
 0.15,
 1e-05,
 1e-05,
 4.0,
 0.46,
 2.0,
 8.5,
 0.47,
 1e-05,
 2.0,
 6.9]

In [26]:
len(new_price)

131

In [27]:
low_noise_data

,houseTypes,area,num_floors,bedrooms,district,city,price,url
68,Bán Nhà,26.0,2.0,2.0,Bình Thạnh,TP Hồ Chí Minh,0.00100,https://batdongsan.vn/nha-dep-tai-binh-thanh-2...
119,Bán Nhà,47.0,2.0,2.0,Tân Phú,TP Hồ Chí Minh,0.00500,https://batdongsan.vn/tay-thanh-ngang-hiem-53m...
134,Bán Nhà,24.0,2.0,2.0,Quận 3,TP Hồ Chí Minh,0.00100,https://batdongsan.vn/khu-vuc-hiem-nha-ban-hem...
184,Bán Nhà,80.0,NaN,NaN,Gò Vấp,TP Hồ Chí Minh,0.00100,https://batdongsan.vn/52-ty-80m2-hem-xe-hoi-go...
216,Bán Nhà riêng,46.0,2.0,2.0,Quận 10,TP Hồ Chí Minh,0.00100,https://batdongsan.vn/ban-nha-46m2-duong-cmt8-...
...,...,...,...,...,...,...,...,...
9146,Bán Nhà mặt phố,NaN,5.0,NaN,Đồng Hới,Quảng Bình,0.00100,https://batdongsan.vn/to-hop-giai-tri-du-lich-...
9275,Bán Nhà riêng,50.0,4.0,3.0,Long Biên,Hà Nội,0.34500,https://batdongsan.vn/ban-nha-ngo-61-nguyen-so...
9285,Bán Nhà mặt phố,70.0,5.0,6.0,Cầu Giấy,Hà Nội,0.42143,https://batdongsan.vn/ban-nha-vip-1-trung-kinh...
9424,Bán Nhà riêng,78.0,1.0,2.0,Biên Hòa,Đồng Nai,0.00100,https://batdongsan.vn/ban-nha-1-tret-1-lau-shr...


In [24]:
low_noise_data.to_excel('low.xlsx', index=False)

In [25]:
hign_noise_data

,houseTypes,area,num_floors,bedrooms,district,city,price,url
113,Bán Nhà riêng,60.0,6.0,4.0,Long Biên,Hà Nội,11500.00,https://batdongsan.vn/nha-dep-ai-mo-bo-de-60m-...
221,Bán Nhà mặt phố,90.0,3.0,NaN,Từ Sơn,Bắc Ninh,6280.00,https://batdongsan.vn/can-ban-nhanh-can-shopho...
300,Bán Nhà mặt phố,162.0,NaN,NaN,Từ Sơn,Bắc Ninh,6000.00,https://batdongsan.vn/shophose-dep-gia-tot-phu...
301,Bán Nhà riêng,223.0,4.0,3.0,Hai Bà Trưng,Hà Nội,3700.00,https://batdongsan.vn/chinh-chu-can-ban-nha-ph...
320,Bán Nhà mặt phố,120.0,5.0,7.0,Tân Bình,TP Hồ Chí Minh,12900.00,https://batdongsan.vn/ban-nha-mat-tien-phan-xa...
...,...,...,...,...,...,...,...,...
9458,Bán Nhà riêng,54.0,4.0,4.0,Gò Vấp,TP Hồ Chí Minh,7300.00,https://batdongsan.vn/ban-nha-dep-4-tang-hem-x...
9499,Bán Nhà riêng,78.0,NaN,5.0,Gò Vấp,TP Hồ Chí Minh,600.00,https://batdongsan.vn/nha-pho-ban-tren-duong-l...
9500,Bán Nhà riêng,68.0,2.0,5.0,Gò Vấp,TP Hồ Chí Minh,58000.00,https://batdongsan.vn/nha-pho-ban-tren-duong-s...
9502,Bán Nhà riêng,83.0,NaN,5.0,Gò Vấp,TP Hồ Chí Minh,54444.44,https://batdongsan.vn/nha-pho-ban-tren-duong-l...
